In [1]:
!pip install --user line-profiler

In [2]:
%load_ext line_profiler

In [1]:
from django.core.files import File
img = ImageFile()
with open('./images/02-NYH-underviser-AN-1.jpg', 'rb') as fp:
    imf=File(fp)
    img.source_file.save('foobar.jpg', imf)


In [2]:
from apps.photo.tasks import post_save_task

In [3]:
post_save_task(img)

15:09:55 2017-04-21 [ INFO] apps.photo.tasks     tasks.py:24   (post_save_task)
	autocrop



img.cropping

In [7]:
import cv2, numpy

In [19]:
def opencv_image(imagedata: bytes, size=400):
    """ Convert ImageFile into a cv2 image for image processing. """

    nparr = numpy.fromstring(imagedata, numpy.uint8)
    cv2img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    cv2img = cv2.cvtColor(cv2img, cv2.COLOR_BGR2RGB)
    width, height = cv2img.shape[0], cv2img.shape[1]
    if width > height:
        height, width = size * height // width, size
    else:
        height, width = size, size * width // height

    cv2img = cv2.resize(cv2img, (height, width))
    return cv2img

with img.source_file as imagefile:
    data = imagefile.read()
    imagefile.close()
d1 = opencv_image(data)

with open('./images/02-NYH-underviser-AN-1.jpg', 'rb') as fp:
    data = fp.read()
d2 = opencv_image(data)

numpy.array_equal(d1, d2)

True

In [27]:

def migrate_cropbox(image_file):
    radius = image_file.crop_diameter / 2
    data = dict(
        left = (image_file.from_left - radius) / 100,
        right = (image_file.from_left + radius) / 100,
        top = (image_file.from_top - radius) / 100,
        bottom = (image_file.from_top + radius) / 100,
        x = (image_file.from_left) / 100,
        y = (image_file.from_top) / 100,
    )
    CropBox = type(image_file.crop_box)
    image_file.crop_box = CropBox(**data)
    image_file.save(update_fields=['crop_box'])

def migrate_image_files(images):
    for im in images:
        migrate_cropbox(im)
        print(im)

# images = ImageFile.objects.order_by('?')[:50]
migrate_image_files(images)

27-motif.jpg
06-KUL-kulsteinorke-01-HBbruk.jpg
29-kulturet-04-CL.jpg
06-Tegnsprak-01-MNP.jpg
10-KUL-litteraturkritikk-02-NWL.jpg
07anmrwanda.jpg
22anmGisli.jpg
16anmklovner.jpg
27-deb-brucewhitelaw-HH-8.jpg
sj-0161092222.jpg
11-MAG-magskinnetbe-12-PSS.jpg
14-helene-uri-02-SE.jpg
anm-salon.jpg
IMG-6802.jpg
06-PLA-stubokonsert-02-SD.jpg
31-nyh-saihphyoephyoe-hh-14.jpg
01-deb-sosos-AN-1.jpg
15-Demoboks-02-KK.jpg
06-NYH-nyhinsidecon-05-MF.jpg
2447110.jpg
15-Kult-blaa7-Web-SL.jpg
12-KUL-khiobarstengtforside-HD.jpg
32-KUL-nyteprofessor-HB-02.jpg
06-NYH-HH-20.jpg
studetid-fadder03-web-KB.jpg
24-NYH-nyhkildesort-03-OD_Hd05ZtS.jpg
BC15964.jpg
vispor-ready.jpg
11-kronikkbilde-web-HBR.jpg
ILLUSTRASJONCV.jpg
F255-Den-egentlige-kommunen-Hardcover-Cover-300dpi-1.jpg
15-MAG-magasinkommu-07-CB.jpg
28-uriks-usa-dk-15jpg-29917976104-o.jpg
21-mag-iss-emimes-rom-3-web-BO.jpg
17-kultur-khioflytting02-KB.jpg
25-KUL-kulnettverks-07-OD.jpg
Foto-Gage-Skidmore-Flickr-CC.jpg
Triggertrigger.jpg
08-lars-mag-making

In [22]:
from line_profiler import LineProfiler

im = ImageFile.objects.order_by('?').first()
lp = LineProfiler()
lp(im.save)()
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.226292 s
File: /app/apps/photo/models.py
Function: save at line 146

Line #      Hits         Time  Per Hit   % Time  Line Contents
   146                                               def save(self, *args, **kwargs):
   147         1            2      2.0      0.0          mtime = self._mtime
   148         1            4      4.0      0.0          self.mtime = None
   149         1        47729  47729.0     21.1          if self.mtime != mtime:  # file changed
   150         1            9      9.0      0.0              self.md5, self.size = None, None
   151         1           29     29.0      0.0          self.mtime
   152         1       117548 117548.0     51.9          self.md5
   153         1        49569  49569.0     21.9          self.size
   154         1        11402  11402.0      5.0          super().save(*args, **kwargs)

